# Run PEST and Parallel PEST
### This should be super easy ~

In [ ]:
import os
import shutil
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
import psutil
import sys
import pyemu
import flopy
import swatmf
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
assert "dependencies" in swatmf.__file__

In [ ]:
from swatmf import swatmf_pst_par, swatmf_pst_utils

# 01. Specify a path to the model folder

In [ ]:
# path to project directory
prj_dir = "D:/spark/pest_webinar"

In [ ]:
main_opt_path = os.path.join(prj_dir, 'main_opt')
os.chdir(main_opt_path)

# 02. Initial run for adjusting weights

In [ ]:
# PEST control file name
pst_name = "mb_zon.pst"

In [ ]:
# initial run
pyemu.os_utils.run(f'pestpp-glm.exe {pst_name}' , cwd=".")

## 02-01 Change weights to make all of observation data visible 

In [ ]:
# reweight
pst = pyemu.Pst(pst_name)
pst.phi

In [ ]:
# you can assign any values to balanced phi value for each group
balanced_groups = {grp:1000 for grp in pst.nnz_obs_groups}
pst.adjust_weights(obsgrp_dict=balanced_groups)

In [ ]:
# Let's create a new control file with the number of iterations set to 30 and incorporate reweighted factors.
pst.control_data.noptmax = 30
pst.write(os.path.join(main_opt_path,'mb_zon_rw.pst'), version=2)

# 03. Perform a history matching in parallel processing

In [ ]:
# check the number of available cores
num_workers = psutil.cpu_count(logical=False)
num_workers

In [ ]:
# set a path to the main directory of model
m_d = os.path.join(prj_dir, "mb_zon_rw_glm")

In [ ]:
os.chdir(prj_dir) # move the loction to project directory
# run PEST in parallel
pyemu.os_utils.start_workers(main_opt_path,"pestpp-glm",'mb_zon_rw.pst',
                            num_workers=num_workers,
                            worker_root='.',
                            master_dir=m_d)

# 04. Let's see how we did:

In [ ]:
os.chdir(m_d)
pst = pyemu.Pst('mb_zon_rw.pst')
pst.phi

In [ ]:
pst.res.loc[pst.nnz_obs_names,:]

In [ ]:
# plot 1 to 1
figs = pst.plot(kind="1to1")

In [ ]:
# plot phi progress
pyemu.plot_utils.phi_progress(pst)